In [1]:
import numpy as np
from scipy.optimize import linprog

In [2]:
def optimize_convex(c, D, c_G, D_G, alpha_G, x_hat, x_wave):
    n, *_ = c.shape
    
    # условие Слейтер
    if not np.any(c_G.dot(x_wave) +\
                  0.5 * x_wave.dot(D_G).dot(x_wave) + alpha_G >= 0):
        raise Exception('Incopatible constraints')

    def F(x):
        return c.dot(x) + 0.5 * x.dot(D).dot(x)

    def f_grad(x):
        return c + D.dot(x)

    # построение целевой функции для ЗЛП 
    fx_wave_grad = f_grad(x_wave)
    active_idxs = np.argwhere(c_G.dot(x_wave) + 0.5 * x_wave.dot(D_G).dot(x_wave) + alpha_G == 0).ravel()

    # построение ограничий
    A = c_G[active_idxs] + D_G[active_idxs].dot(x_wave)
    b = np.zeros(len(active_idxs))
    bounds = np.vstack((np.where(x_wave == 0, 0, None), np.ones(n))).T
    
    res = linprog(fx_wave_grad, A_ub=A, b_ub=b, bounds=bounds)
    l_star = res['x']

    # нахождение alpha
    a = f_grad(x_wave).dot(l_star)
    b = (x_hat - x_wave).dot(f_grad(x_wave))
    
    if b > 0:
        alpha = -a/(2*b)
    else:
        alpha = 1

    def x(t):
        return x_wave + t*(l_star + alpha * (x_hat - x_wave))

    def find_t():
        F_wave = F(x_wave)
        t = 1
        while not (F(x(t)) < F_wave and np.all(c_G.dot(x(t)) + 0.5 * x(t).dot(D_G).dot(x(t)) + alpha_G <= 0)):
            t /= 2
        return t

    return x(find_t())

## Пример

In [3]:
c = np.array([-3, -3])
D = np.array([
    [2, 1],
    [1, 2]
])

######### G ########
c_G = np.array([
    [1, -1],
    [-1, 1]
])
D_G = np.array([
    [[1, 0], [0, 1]],
    [[1, .5], [.5, 1]]
])
alpha_G = np.array([-1, -1.5])

x_hat = np.array([0, 0])
x_wave = np.array([0, 1])

optimize_convex(c, D, c_G, D_G, alpha_G, x_hat, x_wave)

array([ 0.5   ,  0.5625])